In [11]:
import cv2
from PIL import Image
from backbones.iresnet import iresnet100
import torch
import os
import numpy as np
from tqdm import tqdm
from  face_alignment import align

In [12]:
device ='cuda'

net = iresnet100()
model_dict = net.state_dict()
pretrained_dict = torch.load("./models/backbone.pth")
pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
model_dict.update(pretrained_dict)
net.load_state_dict(model_dict)
net.to(torch.device(device))

net.eval()

IResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (prelu): PReLU(num_parameters=64)
  (layer1): Sequential(
    (0): IBasicBlock(
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (prelu): PReLU(num_parameters=64)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (downsample): Sequential(
        (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(2, 2), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): IBas

In [13]:
races = ['African','Asian','Caucasian','Indian']
genders = ['Man','Woman']

In [14]:
African_wom = os.listdir('../images/test/data/African/Woman')
Caucasian_wom = os.listdir('../images/test/data/Caucasian/Woman')
Asian_wom = os.listdir('../images/test/data/Asian/Woman')
Indian_wom = os.listdir('../images/test/data/Indian/Woman')
Asian_man = os.listdir('../images/test/data/Asian/Man')
African_man = os.listdir('../images/test/data/African/Man')
Caucasian_man = os.listdir('../images/test/data/Caucasian/Man')
Indian_man = os.listdir('../images/test/data/Indian/Man')

In [15]:
def to_input(pil_rgb_image):
    np_img = np.array(pil_rgb_image)
    brg_img = ((np_img/ 255.) - 0.5) / 0.5
    tensor = torch.tensor(np.array([brg_img.transpose(2,0,1)])).float()
    return tensor

In [16]:
cache = {}

In [17]:
for race in races:
    with open(f"../images/test/txts/{race}/{race}_pairs.txt", 'r') as f:
        pairs = f.readlines()
    pairs = [pair.strip().split() for pair in pairs]
    for pair in tqdm(pairs):
        try:
            if pair[0] in eval(f'{race}_wom'):
                    gender = "Woman"
            else:
                    gender = "Man"
            sim = 0
            img1 = f"../images/test/data/Just_Images/{race}/{gender}/{pair[0]}_000{pair[1]}.jpg"
            if len(pair) == 3:
                img2 = f"../images/test/data/Just_Images/{race}/{gender}/{pair[0]}_000{pair[2]}.jpg"
                sim = 1
            else:
                if pair[2] in African_wom or pair[2] in Caucasian_wom or pair[2] in Asian_wom or pair[2] in Indian_wom:
                    gen2 = "Woman"
                    if pair[2] in African_wom:
                        race2 = "African"
                    elif pair[2] in Caucasian_wom:
                        race2 = "Caucasian"
                    elif pair[2] in Asian_wom:
                        race2 = "Asian"
                    else:
                        race2 = "Indian"
                else:
                    gen2 = "Man"
                    if pair[2] in African_man:
                        race2 = "African"
                    elif pair[2] in Caucasian_man:
                        race2 = "Caucasian"
                    elif pair[2] in Asian_man:
                        race2 = "Asian"
                    else:
                        race2 = "Indian"
                img2 = f"../images/test/data/Just_Images/{race2}/{gen2}/{pair[2]}_000{pair[3]}.jpg"
            if img1 in cache:
                 vec1 = cache[img1]
            else:
                with torch.no_grad():
                    vec1 = net(to_input(align.get_aligned_face(img1)).to(device))
                vec1 = vec1.cpu().detach().numpy()
                vec1/=np.linalg.norm(vec1)
                cache[img1] = vec1
            if img2 in cache:
                vec2 = cache[img2]
            else:
                with torch.no_grad():
                    vec2 = net(to_input(align.get_aligned_face(img2)).to(device))
                vec2 = vec2.cpu().detach().numpy()          
                vec2/=np.linalg.norm(vec2)
                cache[img2] = vec2
            if img1 not in cache:
                    cache[img1] = vec1
            if img2 not in cache:
                    cache[img2] = vec2
            score = np.dot(vec1, vec2.T)[0][0]
            with open(f"./{race}_{gender}.csv", 'a') as f:
                f.write(f"{img1},{img2},{score},{sim}\n")
        except Exception as e:
            print(pair, e)
            continue

 25%|██▌       | 1525/6000 [00:30<01:21, 55.09it/s]

['m.02r7h1g', '2', 'm.0bk56n', '5'] [Errno 2] No such file or directory: '../images/test/data/Just_Images/African/Man/m.0bk56n_0005.jpg'


100%|██████████| 6000/6000 [01:44<00:00, 57.34it/s]
